In [1]:
import time

import numpy as np
import pandas as pd

from riskmgmt import simulation
from riskmgmt import covariance
from riskmgmt import non_psd_fix
from riskmgmt import var_es

## Test for Covariance and EWCovariance

In [2]:
DailyReturn = pd.read_csv("DailyReturn.csv", index_col= 0)

In [3]:
SPY = DailyReturn["SPY"].to_numpy()[::-1]
AAPL = DailyReturn["AAPL"].to_numpy()[::-1]

In [4]:
m = np.vstack((SPY, AAPL))

In [5]:
covariance.Covar(m)

array([[7.84558898e-05, 9.17763370e-05],
       [9.17763370e-05, 2.57456530e-04]])

In [6]:
covariance.ewCovar(m,0.97)

array([[8.57359775e-05, 1.08880505e-04],
       [1.08880505e-04, 2.73808020e-04]])

## Test for Non-PSD fixes for correlation matrics

In [7]:
n = 500
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 1
sigma[1, 0] = 1

root = non_psd_fix.chol_psd(sigma)
(np.matmul(root, root.T) - sigma).sum()

-7.299705284680158e-11

In [8]:
n = 500
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 0.7357
sigma[1, 0] = 0.7357

W = np.matrix(np.diag(np.full(n,1.0)))

hpsd = non_psd_fix.higham_nearestPSD(sigma)
npsd = non_psd_fix.near_psd(sigma)
norm_hpsd = non_psd_fix.wgtNorm(hpsd - sigma, W)
norm_npsd = non_psd_fix.wgtNorm(npsd - sigma, W)
print("Distance near_psd() = " + str(norm_npsd))
print("Distance higham_nearestPSD() = " + str(norm_hpsd))

Converged in 26 iterations.
Distance near_psd() = 0.39378468350341517
Distance higham_nearestPSD() = 0.008036763056475597


In [9]:
n = 1000
sigma = np.matrix(np.full((n, n), 0.9), dtype='float64')
np.fill_diagonal(sigma, 1)
sigma[0, 1] = 0.7357
sigma[1, 0] = 0.7357

start = time.time()
hpsd = non_psd_fix.higham_nearestPSD(sigma)
end = time.time()
higam_times = end - start

start = time.time()
npsd = non_psd_fix.near_psd(sigma)
end = time.time()
near_times = end - start

print("n=1000")
print("Higam Took: " + str(higam_times) + " seconds")
print("Near_PSD Took: " + str(near_times) + " seconds")

Converged in 26 iterations.
n=1000
Higam Took: 11.860476970672607 seconds
Near_PSD Took: 0.15181803703308105 seconds


## Test for simulation

In [10]:
df = pd.read_csv(r'DailyReturn.csv').iloc[:, 1:]
data = np.transpose(df.to_numpy())
data

array([[ 0.00260826, -0.00103618,  0.00536279, ...,  0.00270354,
        -0.01377859,  0.00040902],
       [ 0.00147385, -0.00528489, -0.00033627, ...,  0.00257024,
        -0.01902807,  0.00511066],
       [ 0.01089746, -0.00514866, -0.00333157, ...,  0.01044504,
        -0.04232256,  0.01771662],
       ...,
       [ 0.01283166, -0.01095702, -0.00017314, ..., -0.00699643,
         0.01212521, -0.01084672],
       [-0.0046963 ,  0.01352631,  0.00201738, ...,  0.01020405,
         0.00663342,  0.00958525],
       [ 0.01284056, -0.00803953,  0.0171446 , ..., -0.01078016,
        -0.01089774, -0.01969426]])

In [11]:
pear_cov = np.matrix(df.cov())
pear_var = df.var()
pear_cor = np.matrix(df.corr())

ewm_cov = np.matrix(covariance.ewCovar(data, 0.97))
ewm_var = np.diag(ewm_cov)
ewm_cor = covariance.getCor(ewm_cov)

pear_var_cor = covariance.getCov(pear_var, pear_cor)
pear_var_ewm_cor = covariance.getCov(pear_var, ewm_cor)
ewm_var_pear_cor = covariance.getCov(ewm_var, pear_cor)
ewm_var_cor = covariance.getCov(ewm_var, ewm_cor)

In [12]:
# Print Function
matrixType = ["EWMA", "EWMA_COR_PEARSON_STD", "PEARSON", "PEARSON_COR_EWMA_STD"]
simType = ["Full", "PCA=1", "PCA=0.75", "PCA=0.5"]

matrix = []
simulation_l = []
runtimes = []
norms = []

nsim = 25000
for sim in simType:
    for mat in matrixType:
        matrix.append(mat)
        simulation_l.append(sim)
        elapse = 0

        if mat == "PEARSON":
            c = pear_var_cor
        elif mat == "EWMA_COR_PEARSON_STD":
            c = pear_var_ewm_cor
        elif mat == "EWMA":
            c = ewm_var_cor
        elif mat == "PEARSON_COR_EWMA_STD":
            c = ewm_var_pear_cor

        if sim == 'Full':
            start = time.time()
            s = simulation.simulateNormal(nsim, c)
            end = time.time()
            elapse = end - start
        elif sim == 'PCA=1':
            start = time.time()
            s = simulation.simulate_pca(c, nsim, 1)
            end = time.time()
            elapse = end - start
        elif sim == 'PCA=0.75':
            start = time.time()
            s = simulation.simulate_pca(c, nsim, 0.75)
            end = time.time()
            elapse = end - start
        elif sim == 'PCA=0.5':
            start = time.time()
            s = simulation.simulate_pca(c, nsim, 0.5)
            end = time.time()
            elapse = end - start


        covar = np.cov(s)
        runtimes.append(elapse)
        norms.append(non_psd_fix.Norms(covar, c))

In [13]:
result = pd.DataFrame(list(zip(matrix, simulation_l, norms, runtimes,)), columns = ['Name','Simulation', 'Norm', 'RunTime'])
result

,Name,Simulation,Norm,RunTime
0,EWMA,Full,5.409863e-07,0.108071
1,EWMA_COR_PEARSON_STD,Full,7.597320e-07,0.104489
2,PEARSON,Full,9.084485e-07,0.097424
3,PEARSON_COR_EWMA_STD,Full,6.820326e-07,0.097441
4,EWMA,PCA=1,4.768120e-03,0.047034
5,EWMA_COR_PEARSON_STD,PCA=1,5.398432e-03,0.044538
6,PEARSON,PCA=1,6.758644e-03,0.072557
7,PEARSON_COR_EWMA_STD,PCA=1,6.089614e-03,0.043286
8,EWMA,PCA=0.75,4.780298e-03,0.015240
9,EWMA_COR_PEARSON_STD,PCA=0.75,5.413316e-03,0.010725


## Test for VaR and ES

In [14]:
df = pd.read_csv("problem1.csv")
df = df - df.mean()
var_norm = var_es.VaR_Norm(df)
es_norm = var_es.ES_Norm(df)
var_t = var_es.VaR_T(df)
es_t = var_es.ES_T(df)
var_his = var_es.VaR_History(df)
es_his = var_es.ES_History(df)

In [15]:
print(f'VaR - Normal = {var_norm*100:.2f}%')
print(f'VaR - T = {var_t*100:.2f}%')
print(f'VaR - History = {var_his*100:.2f}%')
print(f'ES - Normal = {es_norm*100:.2f}%')
print(f'ES - T = {es_t*100:.2f}%')
print(f'ES - History = {es_his*100:.2f}%')

VaR - Normal = 8.13%
VaR - T = 7.57%
VaR - History = 7.46%
ES - Normal = 10.28%
ES - T = 11.39%
ES - History = 11.59%
